# **Extracción ofertas Infojobs**

En este notebook se desarrolla el código para la extracción de la información de los puestos de trabajo en Madrid de : Data science, Data Analyst y Data Engineer.

Se realiza un scrapeo con Selenium en el cual nos loggeamos con un usuario y contraseña en la plataforma e interactuamos como si furamos un humano para evitar así los captchas.

Los datos recogidos se van almacenando en un `csv` dentro de la carpeta `datos_crudo`.

In [1]:
# Módulos de Selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select 

# Otros módulos
import undetected_chromedriver as uc
import os
from dotenv import load_dotenv
import time
import random
from bs4 import BeautifulSoup
import requests as req
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.chrome.options import Options
from zenrows import ZenRowsClient
from datetime import datetime

# Pandas para manejo de datos
import pandas as pd
import numpy as np

from time import sleep

load_dotenv()

True

In [2]:
URL = 'https://www.infojobs.net/'   
usuario = os.getenv('user_infojobs') 
clave = os.getenv('password_infojobs')

Definimos la función la cual se encarga de recoger la información para cada página, entrando en cada oferta, obteniendo la información deseada y almacenándola.

In [3]:
def guardar_ofertas_df(driver, df, empleo):
    
    # Vamos bajando y entrando en cada oferta
    for i in range(1,28):
        print("-------------------------------------------------------")
        try:
            elemento_lista = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="app"]/div/div[3]/div[1]/div[3]/main/ul/li[{i}]')))
            elemento_lista.click()
            time.sleep(random.uniform(3,7))

            contenedor = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#main-wrapper > div > div.container.container-slotbanner > div:nth-child(3) > div.container-expanded.panel-default > div > div.col-8.col-12-medium > div > div.inner.inner-expanded.panel-canvas.panel-rounded')))
            contenedor_texto = contenedor.text

            cabecera = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#main-wrapper > div > div.container.container-slotbanner > div:nth-child(3) > div.panel-canvas.panel-rounded')))
            cabecera_texto = cabecera.text
            print(driver.current_url)
            print(cabecera_texto)

            datos_oferta = {
                        "empleo" : empleo,
                        "url_oferta" : driver.current_url,
                        "cabecera": cabecera_texto,
                        "contenido": contenedor_texto
            }

            df_aux = pd.DataFrame([datos_oferta])
            df = pd.concat([df, df_aux], ignore_index=True)

            df.to_csv(f"datos_crudo/infojobs_{empleo}.csv")

            print(f"Datos de la oferta {i} guardados correctamente.")

            #Volvemos a la pagina principal
            driver.back()
            sleep(random.uniform(3,7))
            driver.execute_script("window.scrollBy(0, 200);")
            sleep(random.uniform(0.3, 0.8))

        except:
            print(f"\nFallo en el elemento de la lista: {i}\n")
            driver.execute_script("window.scrollBy(0, 200);")
            sleep(random.uniform(0.3, 0.8))
            
    return df

A continación se define la función principal la cual ejecuta los siguientes pasos: 
- Accede a la página principal y acepta las cookies si las hubiera.
- Avanza hasta el inicio de sesión y accede con los datos personales.
- Para cada empleo ejecuta un bucle en el que:
    - Navega directamente hasta la url proporcionada en el diccionario para dicho empleo.
    - Ejecuta la función anteriormente comentada.
    - Cmprueba si hay más páginas y en caso de haberla avanza a la siguiente.

In [23]:
driver = uc.Chrome(headless=False,use_subprocess=False)
driver.get(URL)

df = pd.DataFrame()
diccionario_puestos = {
    "data_analyst" : "https://www.infojobs.net/jobsearch/search-results/list.xhtml?keyword=analista+de+datos&normalizedJobTitleIds=&provinceIds=33&countryIds=17&cityId=&searchByType=province",
    "data_science" : "https://www.infojobs.net/ofertas-trabajo?keyword=Data%20scientist&provinceIds=33&sortBy=RELEVANCE&countryIds=17&sinceDate=ANY",
    "data_engineer" : "https://www.infojobs.net/ofertas-trabajo?keyword=Data%20engineer&provinceIds=33&sortBy=RELEVANCE&countryIds=17&sinceDate=ANY"
}

sleep(4)

# intenta encontrar las cookies y aceptar, si no encuentra imprime ya tiene las cookies
try:
    driver.find_element(By.CSS_SELECTOR, '#didomi-popup > div > div')
    driver.execute_script("window.scrollTo(0, 1000);")
    driver.find_element(By.CSS_SELECTOR, '#didomi-notice-agree-button').click()
    
    print('Cookies aceptadas')
    
except Exception as e:
    print(e)
    print('Ya tienes la cookies')


#hacer click en acceso a candidatos
driver.find_element(By.CSS_SELECTOR, '#candidate_login').click()
cuadro_usuario= WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#email')))

for letra in usuario:
    cuadro_usuario.send_keys(letra)
cuadro_clave= driver.find_element(By.CSS_SELECTOR, '#id-password')
for num in clave:
    cuadro_clave.send_keys(num)
cuadro_usuario.submit()
sleep(3)

#Cargamos la página de los trabajos en Madrid
for empleo, url_empleo in diccionario_puestos.items():
    print(f"\n ------------------------BUSCANDO OFERTAS PARA {empleo}---------------------------\n")

    driver.get(url_empleo)
    sleep(random.uniform(3,7))

    pag=1
    
    while True:

        print(f"\n *** \n Página {pag}, filas df: {df.shape[0]}")

        df = guardar_ofertas_df(driver, df, empleo)

        try:
            sleep(15)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            sleep(1)
            driver.execute_script("window.scrollBy(0, -200);")
            sleep(3)

            #OPCION 1

            # try:
            #     driver.find_element(By.XPATH, '//*[@id="app"]/div/div[3]/div[1]/div[3]/main/div[1]/div/ul/li[7]/button').click()
            #     pag +=1

            # except:
            #     siguiente = driver.find_element(By.XPATH, '//*[@id="app"]/div/div[3]/div[1]/div[3]/main/div[1]/div/ul/li[6]/button').text
            #     if siguiente == "SIGUIENTE":
            #         driver.find_element(By.XPATH, '//*[@id="app"]/div/div[3]/div[1]/div[3]/main/div[1]/div/ul/li[6]/button').click()
            #         pag +=1
            #     else:
            #         print("\nNo hay más páginas")
            #         break


            # OPCION 2

            lista_elementos = driver.find_elements(By.CSS_SELECTOR, "ul.sui-MoleculePagination > li")

            if lista_elementos:
                ultimo_elemento = lista_elementos[-1]
                
                texto_boton = ultimo_elemento.text.strip()
                print(texto_boton)

                if texto_boton == "SIGUIENTE":
                    pag +=1
                    boton = ultimo_elemento.find_element(By.TAG_NAME, "button")
                    boton.click()
                else:
                    print("\nNo hay más páginas")
                    break
            else:
                print("No se encontraron elementos en la lista.")

        except:
            print("\n\nNo hay más páginas")
            break

driver.quit()

Cookies aceptadas

 ------------------------BUSCANDO OFERTAS PARA data_engineer---------------------------


 *** 
 Página 1, filas df: 0
-------------------------------------------------------

Falla en el elemento de la lista: 1

-------------------------------------------------------
https://www.infojobs.net/madrid/data-junior-con-frances/of-i7f97b263f8465abe0693d1cb4c22cb?applicationOrigin=search-new%7Celement%7E54064376822&searchId=54064376822&page=3&sortBy=RELEVANCE
GUARDAR
Proceso online
Data Junior con francés
Sopra Steria - Programa Junior para Primer Empleo
Madrid, Madrid (España)
Híbrido
Publicada el 11 de dic
Salario no disponible
Experiencia mínima: no requerida
Tipo de contrato: indefinido, jornada completa
INSCRIBIRME EN ESTA OFERTA
Datos de la oferta 2 guardados correctamente.
-------------------------------------------------------
https://www.infojobs.net/madrid/data-lead/of-ieba43787b2458f931fa5d23e852d8e?applicationOrigin=search-new%7Celement%7E54064376822&searchId=5